In [11]:
from melanet import MelaData
import torch
import torch.nn as nn
import torch.nn.functional as F

## 0 - Load the data

In [25]:
base_dir = '/Users/robinali/Desktop/HEC/2018 - 2019/project/skin-cancer-mnist-ham10000/'

In [37]:
train = MelaData(data_dir=base_dir+'train/', label_csv=base_dir+'HAM10000_metadata.csv')
test = MelaData(data_dir=base_dir+'test/', label_csv=base_dir+'HAM10000_metadata.csv')

In [40]:
bs = 64
train_loader = torch.utils.data.DataLoader(train, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=bs, shuffle=False)

In [36]:
train_loader

## 1 - Basic Neural Network

A basic NN to predict melanomes from raw data

In [12]:
class classifier(nn.Module):
    
    def __init__(self):
        super(classifier, self).__init__()
        # fill the missing entries below
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)
        self.fc = nn.Linear(in_features=128, out_features=2)
        
    def forward(self,x):
        # implement your network here, use F.max_pool2d, F.log_softmax and do not forget to flatten your vector
        x = self.conv1(x)
        x = F.max_pool2d(x,7)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        x = F.log_softmax(x)
        return x

In [13]:
conv_class = classifier()

In [14]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    conv_class = conv_class.cuda()